# Serial Monitor Data Collector

This python scripts below will monitor the given serial port. From that given port it will continously collects the seen data and then saves it in a usable .csv file for us. This is how we collect the necessary sign data from the gloves for our model's training data.

### COM port scanning

This part executes a routine port scan that will print out the available COM ports to chose from.

In [ ]:
import sys
import glob
import serial

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

print(serial_ports())

### Connecting to the COM port & .csv file creation

Below, the serial connection will be established for the given parameters. Also the .csv file will be created initially.

In [ ]:
import serial
import csv

esp32_port = "/dev/tty.usbserial-0001" # The used COM port of the ESP32.
baud_rate = 112500 # The ESP32 runs at 112500 baud rate (can be changed if necessary).
file_name="collected_data.csv" # The name of the .csv file to be generated.

ser = serial.Serial(esp32_port, baud_rate)
print("Connected to ESP32 on port:" + esp32_port)
file = open(file_name, "a")
print("Created file")

### Reading serial COM port

Continously monitoring the serial connection on the given port and while the connection exists and data is being retrieved from it to a local collection.

In [ ]:
sensor_data = [] #store data

# Collect the samples.
while True:
    raw_data = ser.readline()
    data_string = raw_data.decode('utf-8')
    data = data_string[0:][:-2]
    print(data)
    readings = data.split(",")
    sensor_data.append(readings)

### Saving data to the .csv file

The collected data is being written to the created .csv file, hence completing the data collection from the serial port.

In [ ]:
with open(file_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(sensor_data)

print("Data collection complete!")
file.close()